# A Simple Mistral 7B Offline RAG with Reranker

<img src="../images/simple-rag-1-reranker.webp" width=700>

### 1) Load the embedding model and LLM:

In [1]:
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM

import torch

Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Settings.llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=256,
    generate_kwargs={"do_sample": False},
    model_name="mistralai/Mistral-7B-Instruct-v0.2",
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.2",
    device_map="auto",
    model_kwargs={
        "torch_dtype": torch.bfloat16,
        # Since we are using a small GPU with limited memory
        # Set to False if you have a large GPU to speed things up.
        "offload_buffers": True, 
    }
)

Settings.chunk_size = 512

c:\Users\dk\anaconda3\envs\rag\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Users\dk\anaconda3\envs\rag\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Users\dk\anaconda3\envs\rag\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_kwargs" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Users\dk\anaconda3\envs\rag\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_nam

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


### 2) Load data / read in documents

In [2]:
from pathlib import Path
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

data_dir =  Path("..") / "data" / "example-1"

documents = SimpleDirectoryReader(data_dir).load_data()

# Sanity checks
unique_docs = set(d.metadata["file_name"] for d in documents)
print(f"Read documents: {unique_docs}")

Read documents: {'Basic-Scientific-Food-Preparation-Lab-Manual.pdf', 'Basic-Scientific-Food-Preparation-Lab-Manual.txt'}


### 3) Create vector database

- VectorStoreIndex is an in-memory vector database

In [3]:
index = VectorStoreIndex.from_documents(
    documents,
    show_progress=True
)

num_chunks = len(documents)
print(f"Database consists of {num_chunks} chunks")

Parsing nodes:   0%|          | 0/252 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/570 [00:00<?, ?it/s]

c:\Users\dk\anaconda3\envs\rag\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Database consists of 252 chunks


### 4) Set up custom prompt template

In [4]:
from llama_index.core import PromptTemplate

template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n"
)
qa_template = PromptTemplate(template)
query_engine = index.as_query_engine()
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)

### 5) Query the vector database

In [5]:
response = query_engine.query("What is dehydration?")
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Dehydration is a process used to preserve food by removing moisture from it. It can be done through various methods such as sun drying, oven drying, or using a dehydrator. The objective of dehydration is to prevent bacterial growth and extend the shelf life of food. The dehydrated food can be stored in a cool place or rehydrated for consumption. In this lab manual, students are instructed to pretreat, dry, store, and serve some commonly dehydrated fruits and vegetables, and observe the characteristics of various dried fruits. The manual also provides specific directions for pretreating different fruits and vegetables to control enzymatic deterioration before dehydration.


In [6]:
response = query_engine.query("How many tbsp. butter to use for the Cream Puffs?")
print(response)
# Correct answer is 2

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Answer: 2 tbsp. butter are required for the Cream Puffs.


In [7]:
response = query_engine.query(
    "What are the toppings for the Braided Bread "
    "of the Braids, Coffeecake, and Sweet Rolls recipe?"
)
print(response)
# Correct answer is 
# 2 tsp. caraway seeds and ½ cup shredded Cheddar cheese.
# ½ cup diced Swiss cheese and paprika.
# ...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



The toppings for the Braided Bread in the Braids, Coffeecake, and Sweet Rolls recipe are:

1. 2 tsp. caraway seeds and ½ cup shredded Cheddar cheese.
2. ½ cup diced Swiss cheese and paprika.

For the Braided Pineapple Coffeecake, the topping is:

1. ⅓ cup shredded coconut
2. 1 tbsp. melted butter
3. ½ cup well-drained, crushed pineapple
4. 1 egg white
5. 3 tbsp. brown sugar
6. ⅛ tsp. cinnamon

For the Butterscotch Rolls, the topping is:

1. ¼ cup brown sugar
2. 2 tbsp. butter
3. 1 tsp. light corn syrup
